In [ ]:
import nannyml as nml
from IPython.display import display

# Load synthetic data
reference = nml.load_synthetic_binary_classification_dataset()[0]
analysis = nml.load_synthetic_binary_classification_dataset()[1]
display(reference.head())

,distance_from_office,salary_range,gas_price_per_litre,public_transportation_cost,wfh_prev_workday,workday,tenure,identifier,work_home_actual,timestamp,y_pred_proba,period,y_pred
0,5.962247,40K - 60K €,2.119485,8.568058,False,Friday,0.212653,0,1,2014-05-09 22:27:20,0.99,reference,1
1,0.535872,40K - 60K €,2.357199,5.425382,True,Tuesday,4.927549,1,0,2014-05-09 22:59:32,0.07,reference,0
2,1.969519,40K - 60K €,2.366849,8.247158,False,Monday,0.520817,2,1,2014-05-09 23:48:25,1.00,reference,1
3,2.530410,20K - 40K €,2.318722,7.944251,False,Tuesday,0.453649,3,1,2014-05-10 01:12:09,0.98,reference,1
4,2.253635,60K+ €,2.221265,8.884478,True,Thursday,5.695263,4,1,2014-05-10 02:21:34,0.99,reference,1


In [ ]:
print(reference.head().to_markdown(tablefmt="grid"))

+----+------------------------+----------------+-----------------------+------------------------------+--------------------+-----------+----------+--------------+--------------------+---------------------+----------------+-----------+----------+
|    |   distance_from_office | salary_range   |   gas_price_per_litre |   public_transportation_cost | wfh_prev_workday   | workday   |   tenure |   identifier |   work_home_actual | timestamp           |   y_pred_proba | period    |   y_pred |
+====+========================+================+=======================+==============================+====================+===========+==========+==============+====================+=====================+================+===========+==========+
|  0 |               5.96225  | 40K - 60K €    |               2.11948 |                      8.56806 | False              | Friday    | 0.212653 |            0 |                  1 | 2014-05-09 22:27:20 |           0.99 | reference |        1 |
+----+----------

In [ ]:
# Define feature columns
feature_column_names = [
    col for col in reference.columns if col not in [
        'timestamp', 'y_pred_proba', 'period', 'y_pred', 'work_home_actual', 'identifier'
    ]]

calc = nml.DataReconstructionDriftCalculator(
    feature_column_names=feature_column_names,
    timestamp_column_name='timestamp',
    chunk_size=5000
)
calc.fit(reference)
results = calc.calculate(analysis)

In [ ]:
display(results.filter(period='analysis').to_df())

,key,chunk_index,start_index,end_index,start_date,end_date,period,sampling_error,reconstruction_error,alert,upper_threshold,lower_threshold
0,[0:4999],0,0,4999,2017-08-31 04:20:00,2018-01-02 00:45:44,analysis,0.006962,1.118536,False,1.138013,1.096577
1,[5000:9999],1,5000,9999,2018-01-02 01:13:11,2018-05-01 13:10:10,analysis,0.006962,1.115044,False,1.138013,1.096577
2,[10000:14999],2,10000,14999,2018-05-01 14:25:25,2018-09-01 15:40:40,analysis,0.006962,1.125460,False,1.138013,1.096577
3,[15000:19999],3,15000,19999,2018-09-01 16:19:07,2018-12-31 10:11:21,analysis,0.006962,1.128453,False,1.138013,1.096577
4,[20000:24999],4,20000,24999,2018-12-31 10:38:45,2019-04-30 11:01:30,analysis,0.006962,1.122892,False,1.138013,1.096577
5,[25000:29999],5,25000,29999,2019-04-30 11:02:00,2019-09-01 00:24:27,analysis,0.006962,1.228393,True,1.138013,1.096577
6,[30000:34999],6,30000,34999,2019-09-01 00:28:54,2019-12-31 09:09:12,analysis,0.006962,1.220028,True,1.138013,1.096577
7,[35000:39999],7,35000,39999,2019-12-31 10:07:15,2020-04-30 11:46:53,analysis,0.006962,1.237394,True,1.138013,1.096577
8,[40000:44999],8,40000,44999,2020-04-30 12:04:32,2020-09-01 02:46:02,analysis,0.006962,1.206051,True,1.138013,1.096577
9,[45000:49999],9,45000,49999,2020-09-01 02:46:13,2021-01-01 04:29:32,analysis,0.006962,1.242579,True,1.138013,1.096577


In [ ]:
print(results.filter(period='analysis').to_df().to_markdown(tablefmt="grid"))

+----+---------------+---------------+---------------+-------------+---------------------+---------------------+----------+------------------+------------------------+---------+-------------------+-------------------+
|    | key           |   chunk_index |   start_index |   end_index | start_date          | end_date            | period   |   sampling_error |   reconstruction_error | alert   |   upper_threshold |   lower_threshold |
+====+===============+===============+===============+=============+=====================+=====================+==========+==================+========================+=========+===================+===================+
|  0 | [0:4999]      |             0 |             0 |        4999 | 2017-08-31 04:20:00 | 2018-01-02 00:45:44 | analysis |        0.0069621 |                1.11854 | False   |           1.13801 |           1.09658 |
+----+---------------+---------------+---------------+-------------+---------------------+---------------------+----------+-----

In [ ]:
display(results.filter(period='reference').to_df())

,key,chunk_index,start_index,end_index,start_date,end_date,period,sampling_error,reconstruction_error,alert,upper_threshold,lower_threshold
0,[0:4999],0,0,4999,2014-05-09 22:27:20,2014-09-09 08:18:27,reference,0.006962,1.120961,False,1.138013,1.096577
1,[5000:9999],1,5000,9999,2014-09-09 09:13:35,2015-01-09 00:02:51,reference,0.006962,1.118071,False,1.138013,1.096577
2,[10000:14999],2,10000,14999,2015-01-09 00:04:43,2015-05-09 15:54:26,reference,0.006962,1.117237,False,1.138013,1.096577
3,[15000:19999],3,15000,19999,2015-05-09 16:02:08,2015-09-07 07:14:37,reference,0.006962,1.125514,False,1.138013,1.096577
4,[20000:24999],4,20000,24999,2015-09-07 07:27:47,2016-01-08 16:02:05,reference,0.006962,1.109446,False,1.138013,1.096577
5,[25000:29999],5,25000,29999,2016-01-08 17:22:00,2016-05-09 11:09:39,reference,0.006962,1.122759,False,1.138013,1.096577
6,[30000:34999],6,30000,34999,2016-05-09 11:19:36,2016-09-04 03:30:35,reference,0.006962,1.107138,False,1.138013,1.096577
7,[35000:39999],7,35000,39999,2016-09-04 04:09:35,2017-01-03 18:48:21,reference,0.006962,1.127134,False,1.138013,1.096577
8,[40000:44999],8,40000,44999,2017-01-03 19:00:51,2017-05-03 02:34:24,reference,0.006962,1.114237,False,1.138013,1.096577
9,[45000:49999],9,45000,49999,2017-05-03 02:49:38,2017-08-31 03:10:29,reference,0.006962,1.110450,False,1.138013,1.096577


In [ ]:
print(results.filter(period='reference').to_df().to_markdown(tablefmt="grid"))

+----+---------------+---------------+---------------+-------------+---------------------+---------------------+-----------+------------------+------------------------+---------+-------------------+-------------------+
|    | key           |   chunk_index |   start_index |   end_index | start_date          | end_date            | period    |   sampling_error |   reconstruction_error | alert   |   upper_threshold |   lower_threshold |
+====+===============+===============+===============+=============+=====================+=====================+===========+==================+========================+=========+===================+===================+
|  0 | [0:4999]      |             0 |             0 |        4999 | 2014-05-09 22:27:20 | 2014-09-09 08:18:27 | reference |        0.0069621 |                1.12096 | False   |           1.13801 |           1.09658 |
+----+---------------+---------------+---------------+-------------+---------------------+---------------------+-----------+

In [ ]:
figure = results.plot(plot_reference=True)
figure.show()

In [ ]:
figure.write_image('../_static/drift-guide-multivariate.svg')


In [ ]:
# Define feature columns
feature_column_names = [
    col for col in reference.columns if col not in [
        'timestamp', 'y_pred_proba', 'period', 'y_pred', 'work_home_actual', 'identifier'
    ]]

from sklearn.impute import SimpleImputer

calc = nml.DataReconstructionDriftCalculator(
    feature_column_names=feature_column_names,
    timestamp_column_name='timestamp',
    chunk_size=5000,
    imputer_categorical=SimpleImputer(strategy='constant', fill_value='missing'),
    imputer_continuous=SimpleImputer(strategy='median')
)
calc.fit(reference)
results = calc.calculate(analysis)
